In [1]:
import numpy as np
import random

# Define Rastrigin Function

In [2]:
def rastrigin(X):
    if isinstance(X[0], (int, float)):
        X = [[X[i]] for i in range(len(X))]
    
    val = []
    for xi in X:
        fx = 10 * len(xi) + sum(np.array(xi) ** 2 - 10 * np.cos(2 * np.pi * np.array(xi)))
        val.append(fx)
    return np.array(val)

# Generate Primary Target Vectors

In [3]:
def generate_target(num_variables,population_size,x_min,x_max):
    return np.random.uniform(x_min,x_max,(population_size,num_variables))   

# Differential Evolution

In [5]:
def diff_evol(num_iterations, population_size, num_variables, x_min, x_max, scale_factor, crossover_probability):
    target = generate_target(num_variables, population_size, x_min, x_max)
    trial = np.zeros_like(target)
    
    for i in range(num_iterations):
        mutant = np.clip(np.array([random.sample(list(target), 3)[0] + scale_factor * (random.sample(list(target), 3)[1] - (random.sample(list(target), 3)[2])) for _ in range(population_size)]), x_min, x_max)
        
        for j in range(population_size):  
            I_rand = np.random.randint(0, num_variables)
            for k in range(num_variables):  
                if np.random.uniform(0, 1) <= crossover_probability or k == I_rand:
                    trial[j, k] = mutant[j, k]
                else:
                    trial[j, k] = target[j, k]
        
        target_dict = {tuple(target[i]): rastrigin(target)[i] for i in range(population_size)}
        trial_dict = {tuple(trial[i]): rastrigin(trial)[i] for i in range(population_size)}
        
        target_dict.update(trial_dict)
        items = target_dict.items()
        items = sorted(items, key=lambda x: x[1])
        new_target = [np.array(item[0]) for item in items[:population_size]]
        
        target = np.array(new_target)
    
    return target[0]

In [11]:
soln = diff_evol(100, 50, 5, -5.12, 5.12, .5, .5)

In [14]:
soln

array([-6.40152422e-06, -1.91944170e-06,  4.18748740e-06,  4.31676670e-06,
       -1.05047741e-06])

In [13]:
rastrigin([soln])

array([1.6255612e-08])